In [1]:
import numpy as np
from qutip import *

# --------------------------
# Platform scaling (pick one)
# --------------------------
PLATFORM = "quera"   # "ibm" or "quera"

def platform_params(platform="ibm"):
    if platform.lower() == "ibm":
        # IBM-style transmon (rad/s, seconds)
        omega_max = 2*np.pi*12.5e6      # 40 ns pi pulse -> 12.5 MHz
        T_max     = 0.32e-6             # ~8*pi/omega_max; use 1e-6..2e-6 for very adiabatic
        Ep        = 2*np.pi*100e6       # penalty ~100 MHz
        time_unit_label = "time [µs]"
        energy_unit_label = "energy [MHz]"  # plots will convert rad/s -> MHz
        to_time_units = 1e6             # s -> µs
        to_freq_units = 1/(2*np.pi*1e6) # rad/s -> MHz
    elif platform.lower() == "quera":
        # QuEra Aquila (rad/µs, microseconds)
        omega_max = 10.0                # <= 15.8 rad/µs
        T_max     = 1.9                 # µs
        Ep        = 30.0                # rad/µs (>> omega_max)
        time_unit_label = "time [µs]"
        energy_unit_label = "energy [MHz]"  # convert rad/µs -> MHz
        to_time_units = 1.0             # µs -> µs
        to_freq_units = 1/(2*np.pi)     # rad/µs -> MHz
    else:
        raise ValueError("PLATFORM must be 'ibm' or 'quera'")
    return dict(omega_max=omega_max, T_max=T_max, Ep=Ep,
                time_unit_label=time_unit_label,
                energy_unit_label=energy_unit_label,
                to_time_units=to_time_units,
                to_freq_units=to_freq_units)

params = platform_params(PLATFORM)
omega_max = params["omega_max"]
T_max     = params["T_max"]
Ep        = params["Ep"]

# --------------------------
# Single-qubit primitives
# --------------------------
I = qeye(2)
X = sigmax()
Y = sigmay()
Z = sigmaz()

# --------------------------
# Logical operators (3-qubit repetition, bit-flip code)
# Code space: |0_L>=|000>, |1_L>=|111>
# Stabilizers for X-error protection: Z1Z2 and Z2Z3
# --------------------------
X_L = tensor(X, X, X)
Z_L = tensor(Z, Z, Z)              # acts nontrivially on code; anticommutes with X_L
I_L = tensor(I, I, I)

S1 = tensor(Z, Z, I)               # Z1 Z2
S2 = tensor(I, Z, Z)               # Z2 Z3
S_list = [S1, S2]

# Penalty Hamiltonian: -Ep * (S1 + S2)
Hp = -Ep * sum(S_list, 0*I_L)      # 'start' ensures Qobj sum

# --------------------------
# Basis & logical states (optional, for checks)
# --------------------------
basis_states = [basis(2, 0), basis(2, 1)]
logical_zero = tensor(basis_states[0], basis_states[0], basis_states[0])
logical_one  = tensor(basis_states[1], basis_states[1], basis_states[1])

# --------------------------
# RAP pulses
# --------------------------
def omega_t(t, T_max=T_max, omega_max=omega_max):
    return omega_max * np.sin(np.pi * t / T_max)

def delta_t(t, T_max=T_max, omega_max=omega_max):
    return -omega_max * np.cos(np.pi * t / T_max)

def H(t):
    # Logical RAP drive
    return X_L * omega_t(t) + Z_L * delta_t(t) + Hp

# --------------------------
# Time grid
# --------------------------
def time_list(num_points=101):
    return np.linspace(0, T_max, num_points)

t_list = time_list()

# --------------------------
# Spectrum vs. time with simple eigenstate matching
# --------------------------
def spectrum_track(t_list, n_track=2):
    """
    Track the lowest 'n_track' eigenstates by overlap with t=0 eigenstates.
    Returns: dict {idx: energies_over_time}
    """
    # t=0 diagonalization
    evals0, estates0 = H(0).eigenstates()
    dim = len(evals0)
    energies = {i: [evals0[i]] for i in range(dim)}

    for k in range(1, len(t_list)):
        t = t_list[k]
        evals, estates = H(t).eigenstates()

        # Track the first n_track states by overlap with initial estates0
        used = set()
        for i in range(n_track):
            ref = estates0[i]
            best_j, best_over = 0, -1.0
            for j, psi in enumerate(estates):
                if j in used:
                    continue
                ov = abs(ref.overlap(psi))**2
                if ov > best_over:
                    best_over, best_j = ov, j
            energies[i].append(evals[best_j])
            used.add(best_j)

        # Append the rest by index (no tracking)
        for i in range(n_track, dim):
            energies[i].append(evals[i])

    return energies, params

energies, p = spectrum_track(t_list, n_track=2)

# --------------------------
# Plot with Plotly (kept your style; kaleido optional)
# --------------------------
import plotly.graph_objects as go
import os

# Pulses
omega_vals = [omega_t(t) for t in t_list]
delta_vals = [delta_t(t) for t in t_list]

# Convert to nice plot units
t_plot   = t_list * p["to_time_units"]
w_plot   = np.array(omega_vals) * p["to_freq_units"]
d_plot   = np.array(delta_vals) * p["to_freq_units"]

os.makedirs('pngs', exist_ok=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t_plot, y=w_plot, mode='lines', name='Ω(t)'))
fig.add_trace(go.Scatter(x=t_plot, y=d_plot, mode='lines', name='Δ(t)', line=dict(dash='dash')))
fig.update_layout(
    title=f"RAP pulses ({PLATFORM.upper()})",
    xaxis_title=p["time_unit_label"],
    yaxis_title="amplitude [MHz]",
    template="simple_white"
)
try:
    fig.write_image('pngs/omega_delta.png', scale=2, width=800, height=500)
except Exception as e:
    pass
fig.show()

# Spectrum figure
fig2 = go.Figure()
colors_main = ['red', 'blue']
for i, e_list in energies.items():
    e_plot = np.array(e_list) * p["to_freq_units"]
    if i < 2:
        fig2.add_trace(go.Scatter(x=t_plot, y=e_plot, mode='lines',
                                  line=dict(width=3, color=colors_main[i%2]),
                                  name=f"tracked state {i}"))
    else:
        fig2.add_trace(go.Scatter(x=t_plot, y=e_plot, mode='lines',
                                  line=dict(width=1, dash='dash'),
                                  name=f"state {i}"))
fig2.update_layout(
    title=f"Instantaneous spectrum vs time ({PLATFORM.upper()})",
    xaxis_title=p["time_unit_label"],
    yaxis_title=p["energy_unit_label"],
    template="simple_white",
    legend_title=""
)
try:
    fig2.write_image('pngs/spectrum.png', scale=2, width=900, height=600)
except Exception as e:
    pass
fig2.show()
